In [3]:
def egcd(a, b):
    x0, x1, y0, y1 = 0, 1, 1, 0
    while a != 0:
        #(q, a), b = divmod(b, a), a
        q, a, b = b // a, b % a, a
        y0, y1 = y1, y0 - q * y1
        x0, x1 = x1, x0 - q * x1
    return b, x0, y0

def gcd(x, y):
    res = 0
    if x > y:
        small = y
    else:
        small = x
    for i in range(1, small + 1):
        if (x % i == 0) and (y % i == 0):
            res = i

    return res

import secrets
import string


def genrandomstring(length):
    password_characters = string.ascii_letters + string.digits + string.punctuation + string.whitespace
    password = ''.join(secrets.choice(password_characters) for i in range(length))
    return password

def modinv(a, m):
    g, x, y = egcd(a, m)
    if g != 1:
        raise Exception('modular inverse does not exist')
    else:
        return x % m
    
def modpow(a, n, p):
    res = 1
    a = a % p

    while n > 0:
        if n % 2:
            res = (res * a) % p
            n = n - 1
        else:
            a = (a ** 2) % p
            n = n // 2

    return res % p


def getrandom(size):
    prime = secrets.randbits(size)
    return prime


def randomexponent(phi):
    e = 0
    good = False
    while not good:
        e = getrandom(16)
        e = e % 19193
        e += 46341
        e += 1 - e % 2
        if e > 65535:
            e = 46341
        while gcd(e, phi) != 1:
            e = e + 2
        if e <= phi:
            good = True
    return e


In [4]:
def fermatprimalitytest(n, k):
    if n == 1 or n == 4:
        return False
    if n == 2 or n == 3:
        return True

    if n % 2 == 0:
        return False

    for i in range(k):
        a = secrets.randbelow(n - 1)
        if a < 1:
            a = 1

        #   if gcd(n, a) != 1:
            #   return False

        if modpow(a, n - 1, n) != 1:
            return False

    return True

def millerwitness(d, n):
    if n < 4:
        return False
    a = secrets.randbelow(n - 2)
    if a < 2 and n > 3:
        a = a + 2
    if a < 1:
        a = 1

    x = modpow(a, d, n)

    if x == 1 or x == n - 1:
        return True

    while d != n - 1:
        x = (x * x) % n
        d *= 2

        if x == 1:
            return False
        if x == n - 1:
            return True
    return False



def millerrabinprimalitytest(n, k):
    # Corner cases
    if n <= 1 or n == 4:
        return False
    if n <= 3:
        return True

    # Find r such that n = 2^d * r + 1 for some r >= 1
    d = n - 1
    while d % 2 == 0:
        d //= 2

    for i in range(k):
        if not millerwitness(d, n):
            return False

    return True



def randomprime():
    # random number between 46341 and 65535
    prime = getrandom(16)
    prime = prime % 19193
    prime += 46341
    prime += 1 - prime % 2
    if prime > 65521:
        prime = 65521
    while not fermatprimalitytest(prime, 40) or not millerrabinprimalitytest(prime, 40):
        prime = prime + 2
    return prime

In [5]:
def rsaencrypt(plaintxt, pubkey, isint):
    if isint:
        cipher = [modpow(char, pubkey.exp, pubkey.mod) for char in plaintxt]
    else:
        cipher = [modpow(ord(char), pubkey.exp, pubkey.mod) for char in plaintxt]

    return cipher

In [6]:
class PublicKey:
    def __init__(self, exp, mod):
        self.exp = exp
        self.mod = mod
        
class PrivateKey:
    def __init__(self, modinv, mod):
        self.modinv = modinv
        self.mod = mod

In [15]:
def commonmodulus():
    plaintxt = genrandomstring(128)
    
    
    p1 = randomprime()
    p2 = randomprime()
    while p1 == p2:
        p2 = randomprime()
    print("Random prime numbers are: ", p1, "and", p2, '\n')
    
    
    phi = (p1 - 1) * (p2 - 1)
    e1 = randomexponent(phi)
    e2 = randomexponent(phi)
    while e1 == e2 != 1 or egcd(e1, e2)[0] != 1 or egcd(e1, p1*p2)[0] != 1 or egcd(e2, p1*p2)[0] != 1:
        e2 = randomexponent(phi)
    print("Exponent 1 is", e1, "\nExponent 2 is", e2, '\n')
    
    
    mod = p1 * p2
    print("Modulo is: ", mod, '\n')
    
    
    print("Plain text:")
    print(plaintxt, "\n")
    
    
    c1 = rsaencrypt(plaintxt, PublicKey(e1, mod), False)
    c2 = rsaencrypt(plaintxt, PublicKey(e2, mod), False)
    #print("Cipher text 1 = ", c1, '\n')
    #print("Cipher text 2 = ", c2, '\n')
    
    
    x = modinv(e1, e2)
    #y = (gcd(e1, e2) - e1 * x) / e2
    y = (1 - e1 * x) / e2
    result = ""
    for i in range(0, len(c1)):
        temp = modinv(c2[i], mod)
        #print("[ATTACK] modinv c =", temp)
        m1 = modpow(c1[i], x, mod)
        #print("[ATTACK] exp mod c1^x =", m1)
        m2 = modpow(temp, -y, mod)
        #print("[ATTACK] exp mod temp-y =", m2)
        result = result + chr((m1 * m2) % mod)
        

    print("Deciphered text:")
    print(result)


In [17]:
commonmodulus()

Random prime numbers are:  48673 and 47543 

Exponent 1 is 48797 
Exponent 2 is 59909 

Modulo is:  2314060439 

Plain text:
JJA3wj]\8w<	; & l9 uI`HMdL75K`o>3hZj7XZ4` ]Mc_!Z6 >m-Tz+[`i71?Z|=pK{Y1k	t5~/G0Eh=76Z@&O1 

Deciphered text:
JJA3wj]\8w<	; & l9 uI`HMdL75K`o>3hZj7XZ4` ]Mc_!Z6 >m-Tz+[`i71?Z|=pK{Y1k	t5~/G0Eh=76Z@&O1
